# CSC/XMM validation catalogs

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import matplotlib.pyplot as plt
# plt.style.use('seaborn')

import seaborn as sns

from scripts.utils import set_mpl, data_path
from scripts.cross_match_scripts import cross_match_data_frames,  add_separation_columns
set_mpl()

%matplotlib inline
from astropy.table import Table

matplotlib settings set


matplotlib settings set
matplotlib settings set


***

In [3]:
ero_df = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14.pkl') 
ero_df = ero_df.query('DET_LIKE_0>10')
ero_df.sample(5)

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH
6641,SRGe J103902.8+555443,159.761483,55.911847,153.269183,52.703133,9.240471,10.188679,4.299193e-15,1.343206e-15,20.465979,6.394230,4349.036621,0.0,0.0,0.0,8627,7475,159.763492,55.910558,4.045943,120.501244,0.000010,6.268413e+08,6.270701e+08,0.0,0.0,0.004706,0.001470,19.075880,159.761483,55.911847,853632377942267008,7.702683,1,1,17.833529,-1.0,0,SDSS J103902.71+555444.1,2.042329,QSO,7.586270,1,0,0,0,15.244000,15.215000,12.754,30.2,14.2,0.0,1,3,1,1237657222559629409,0,4.330578,70845965876,4.045943,3.303531,4.986985,8.086207,136.028705,43.056168,3.308397e-15,5.426967e+19
5793,SRGe J104418.6+562638,161.077391,56.443843,151.830489,52.955952,7.232556,16.620613,5.446213e-15,1.437604e-15,25.450989,6.718142,4269.308594,0.0,0.0,0.0,7758,6623,161.079400,56.442554,3.121623,127.636147,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.005961,0.001574,19.118587,161.077391,56.443843,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,0,16.926001,17.209999,12.536,11.6,2.7,1.0,2,1,1,1237657772323832330,0,3.730241,68714395926,3.121623,2.585688,3.903334,6.329109,136.514388,43.889983,4.191073e-15,5.885968e+19
3664,SRGe J104610.8+585518,161.544816,58.921586,148.485229,51.538673,5.801307,33.968182,8.770101e-15,1.764666e-15,41.394985,8.329245,4312.118164,0.0,0.0,0.0,3728,2979,161.546824,58.920296,2.451130,54.239338,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.009600,0.001932,18.436668,161.544816,58.921586,-1,-1.000000,0,-1,NaN,-1.0,-1,[VV2010c] J104611.0+585515,1.990000,AGN,4.501851,1,1,1,0,17.518999,17.142000,12.389,6.8,0.8,0.6,1,1,0,1237655109446534041,0,7.880840,59183101693,2.451130,2.074007,3.130904,5.076643,134.944668,46.119427,6.748935e-15,8.269307e+19
901,SRGe J103928.2+570545,159.867529,57.095831,151.669595,52.031276,3.462681,178.782913,2.727783e-14,2.795898e-15,128.881393,13.209967,4316.461426,0.0,0.0,0.0,1021,828,159.869537,57.094541,1.291740,57.355171,0.000010,6.268413e+08,6.270701e+08,0.0,0.0,0.029858,0.003060,19.965200,159.867529,57.095831,854347751991489920,7.605225,1,0,20.571882,-1.0,0,SDSS J103928.12+570546.4,0.721768,QSO,7.595796,1,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,2,0,1237655107835265043,0,5.122304,66143896217,1.291740,1.237932,1.868772,3.030144,135.255535,44.100944,2.099136e-14,5.408834e+19
3260,SRGe J104843.8+575632,162.182490,57.942342,149.306722,52.441147,6.401806,38.234032,9.829562e-15,1.831088e-15,46.159981,8.598857,4290.213867,0.0,0.0,0.0,3410,2714,162.184499,57.941052,2.734193,71.667953,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.010759,0.002004,24.439270,162.182490,57.942342,854446330080084352,4.091991,1,0,21.082558,-1.0,0,SDSS J104843.96+575631.0,1.114418,QSO,4.091777,1,1,0,1,16.132999,15.183000,12.222,21.0,15.6,3.7,1,2,1,1237655108372791552,9215529894842486784,4.070108,62871963307,2.734193,2.288689,3.454987,5.602131,136.108575,45.477622,7.564231e-15,8.214656e+19


In [4]:
# desi_lh.gz_pkl number of sources and area
false_dens_arcmin = 2418574 / (41.729 * 3600)
print(false_dens_arcmin)
false_dens_arcsec = false_dens_arcmin / 3600

print(f'{ false_dens_arcsec:.3f}')

FALSE_PROB = .03



def fsource_r(rho: float, f_thresh: float) -> float:
    """
    The function calculates the radius which corresponds to the probability `f_thresh`
    to FIND one or more false sources. 

    Args:
        rho (float): the density of the false sources (in arcmin^{-2}).
        f_thresh (float): probability to FIND one or more false sources.

    Returns:
        float: radius (in arcsec).
    """

    rho = rho / 3600 # arcmin^{-2} -> arcsec^{-2}
    pf_r = np.sqrt(-1 / (rho * np.pi) * np.log(1 - f_thresh))
    
    return pf_r


def find_secure_hostless(erosita, x_ray, desi, x_ray_name):

    ero_x_orig = cross_match_data_frames(erosita, x_ray,
    'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = x_ray_name)
    
    ero_x = ero_x_orig.query(f"{x_ray_name}_n_near==1 & {x_ray_name}_n_matches==1")

    ero_x = ero_x.query(f"{x_ray_name}_secure==True")
    print('SECURITY FILTER APPLIED, ', x_ray_name)
    print('cross-matche: only one X source within 30 arcsec and it is unique for the eROSITA source', ero_x.shape[0])
    assert len(ero_x) == ero_x[f'{x_ray_name}_name'].nunique()

    x_desi_all = cross_match_data_frames(ero_x, desi, colname_ra1 = f'{x_ray_name}_ra',colname_dec1 = f'{x_ray_name}_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')


    x_desi_closest = (x_desi_all
        .loc[x_desi_all.groupby(f'{x_ray_name}_name')['desi_sep'].idxmin()]
        )

    x_hostless = x_desi_closest.query(f"desi_sep>2*{x_ray_name}_r_98 & desi_sep>2*r_false")

    print(f'hostless {x_ray_name} near erosita sources', x_hostless.shape[0])



    return x_hostless





r_false = fsource_r(false_dens_arcmin, FALSE_PROB)
print(f'Радиус для {FALSE_PROB:.0%} вероятности найти ложный: {r_false:.2f}"')
ero_df['r_false'] = r_false


desi = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')
desi.sample(5)

16.099741453452303
0.004
Радиус для 3% вероятности найти ложный: 1.47"


,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,type,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,mjd_max,mjd_min,iso_max,iso_min,desi_id
1243725,9011,903,609938,157.425220,57.460025,0.014783,0.162046,0.511693,4.757088,4.175108,40.798237,777.83780,826.36520,256.604370,184.217250,4.467131,1.103748,0.001525,0.000019,27.052158,24.460112,23.218641,20.805305,20.947506,18.473220,15.272711,0.424956,2.595790,6.945033,10.054385,4.386343,1.593429,3.356615,PSF,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,NaN,0,58194.392789,57787.380856,2018-03-17 09:25:36.961,2017-02-03 09:08:26.000,9011_609938_903
1672120,9011,540,605187,161.796406,55.907205,209.653610,333.961500,359.460720,99.683800,50.990086,81.032330,-347.77188,28.83458,5.754375,5.936611,2.146548,0.830451,0.001356,0.000017,16.668259,16.171907,16.100327,17.501837,18.230300,17.728144,NaN,1125.794600,801.116100,875.831850,146.047760,46.466790,2.984377,-1.418373,PSF,0.215833,214.50784,6.651292,183.31306,-4.307619,120.08669,G2,850955586820528768,57902.156667,57497.235243,2017-05-29 03:45:36.000,2016-04-19 05:38:45.000,9011_605187_540
981661,9011,1311,609174,164.943578,57.272428,0.107073,0.289606,0.818389,0.640734,1.268480,5.467985,-114.11282,294.32300,103.874500,99.858154,4.898828,1.211459,0.001552,0.000018,24.901990,23.829440,22.708628,22.981943,22.240953,20.655252,NaN,1.836922,2.951633,8.178080,1.418155,1.396169,0.215421,-0.487103,REX,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,NaN,0,58194.410133,57787.390301,2018-03-17 09:50:35.462,2017-02-03 09:22:02.000,9011_609174_1311
1002814,9011,1834,609172,164.105812,57.302953,0.396762,0.495593,1.507899,2.516898,2.291780,-8.223105,79.08155,223.16553,92.182610,30.004698,4.297351,1.051345,0.001106,0.000014,23.478092,23.244955,22.044430,21.496370,21.598667,NaN,17.754740,5.927124,4.758276,8.259748,5.217544,2.349880,-0.273490,0.297248,REX,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,NaN,0,58194.410133,57737.529606,2018-03-17 09:50:35.462,2016-12-15 12:42:38.000,9011_609172_1834
734792,9011,2050,614521,163.907196,58.905267,0.283648,0.465302,0.352986,1.460292,0.525638,1.401384,-315.07336,629.37410,257.312740,64.167885,4.755789,1.125817,0.001062,0.000013,23.840317,23.311980,23.620155,22.087313,23.197310,22.133400,NaN,7.115964,7.463898,2.827592,3.184572,0.557725,0.045659,-1.130143,PSF,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,NaN,0,58194.407238,57837.296689,2018-03-17 09:46:25.360,2017-03-25 07:07:13.949,9011_614521_2050


In [5]:
csc_df = pd.read_pickle(data_path+'csc_all_lh.pkl')
csc_desi = find_secure_hostless(ero_df, csc_df, desi, 'csc')
csc_desi['csc_ero_flux_ratio'] = csc_desi['csc_flux_05_2'] / csc_desi['flux_05-20']


cross-match radius 30 arcsec
total matches: 777 out of 6885 x 2329
	 total unique pairs: 722
	 total non-unique pairs (duplicates in df2): 55
SECURITY FILTER APPLIED,  csc
cross-matche: only one X source within 30 arcsec and it is unique for the eROSITA source 461
cross-match radius 15 arcsec
total matches: 2093 out of 461 x 2418574
	 total unique pairs: 2093
	 total non-unique pairs (duplicates in df2): 0
hostless csc near erosita sources 13


***

In [6]:
xmm_df = pd.read_pickle(data_path+'xmm_all_lh.pkl')
xmm_df.rename(columns={'sc_ra':'ra', 'sc_dec':'dec', 'srcid':'name'}, inplace=True)
xmm_desi = find_secure_hostless(ero_df, xmm_df, desi, 'xmm')
xmm_desi['xmm_ero_flux_ratio'] = xmm_desi['xmm_flux_05_2'] / xmm_desi['flux_05-20']


cross-match radius 30 arcsec
total matches: 839 out of 6885 x 1791
	 total unique pairs: 817
	 total non-unique pairs (duplicates in df2): 22
SECURITY FILTER APPLIED,  xmm
cross-matche: only one X source within 30 arcsec and it is unique for the eROSITA source 646
cross-match radius 15 arcsec
total matches: 2831 out of 646 x 2418574
	 total unique pairs: 2831
	 total non-unique pairs (duplicates in df2): 0
hostless xmm near erosita sources 23


## Cross-correlate eROSITA with XMM and Chandra

so that for every eROSITA source we have a number of neighbors in XMM and Chandra. 

In [7]:
ero_csc_n_neigh = cross_match_data_frames(ero_df, csc_df,
 'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'csc')
n_chandra_dict = dict(ero_csc_n_neigh.groupby('srcname_fin')['csc_sep'].count())

ero_xmm_n_neigh = cross_match_data_frames(ero_df, xmm_df,
    'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'xmm')
n_xmm_dict = dict(ero_xmm_n_neigh.groupby('srcname_fin')['xmm_sep'].count())

cross-match radius 30 arcsec
total matches: 777 out of 6885 x 2329
	 total unique pairs: 722
	 total non-unique pairs (duplicates in df2): 55
cross-match radius 30 arcsec
total matches: 839 out of 6885 x 1791
	 total unique pairs: 817
	 total non-unique pairs (duplicates in df2): 22


# Joining CSC and XMM counterparts

In [8]:
print('XMM companions', xmm_desi.shape[0])
print('CSC companions', csc_desi.shape[0])
print('Total companions possible', csc_desi.shape[0] + xmm_desi.shape[0])
xmm_desi = xmm_desi.query("xmm_flux_05_2/`flux_05-20`<5 and xmm_flux_05_2/`flux_05-20`>0.2")
csc_desi = csc_desi.query("csc_flux_05_2/`flux_05-20`<5 and csc_flux_05_2/`flux_05-20`>0.2")

print('XMM companions after flux cut', xmm_desi.shape[0])
print('CSC companions after flux cut', csc_desi.shape[0])
print('Total companions possible after flux cut', csc_desi.shape[0] + xmm_desi.shape[0])

xmm_desi.rename(columns={'desi_desi_id':'desi_id'}, inplace=True)
csc_desi.rename(columns={'desi_desi_id':'desi_id'}, inplace=True)

XMM companions 23
CSC companions 13
Total companions possible 36
XMM companions after flux cut 22
CSC companions after flux cut 10
Total companions possible after flux cut 32


In [10]:
final_ctps = pd.concat([xmm_desi, csc_desi])

final_ctps['n_csc_neigh'] = final_ctps['srcname_fin'].map(n_chandra_dict, na_action='ignore')
final_ctps['n_xmm_neigh'] = final_ctps['srcname_fin'].map(n_xmm_dict, na_action='ignore')
final_ctps['n_xmm_neigh'] = final_ctps['n_xmm_neigh'].fillna(0)
final_ctps['n_csc_neigh'] = final_ctps['n_csc_neigh'].fillna(0)

final_ctps.sort_values(by='srcname_fin', inplace=True)
final_ctps.reset_index(drop=True, inplace=True)


n_desi_ctps = final_ctps.groupby('srcname_fin')['desi_id'].transform(lambda x: len(x.unique()))
final_ctps['n_desi_ctps'] = n_desi_ctps


final_ctps = final_ctps.query('n_desi_ctps == 1')


final_ctps.drop_duplicates(subset=['srcname_fin', 'desi_id'], inplace=True)


final_ctps = add_separation_columns(final_ctps, colname_ra1 = 'RA_fin', colname_dec1 = 'DEC_fin', colname_ra2 = 'desi_ra', colname_dec2 = 'desi_dec', colname = 'sep_ero_desi')
print('After merging XMM and CSC and deleting duplicates we have ', final_ctps.shape[0], ' sources')

final_ctps = final_ctps.query(" ( (n_csc_neigh==0 & n_xmm_neigh==1) | (n_csc_neigh==1 & n_xmm_neigh==0) | (n_csc_neigh==1 & n_xmm_neigh==1) )")
print('After removing sources with more than one X-ray neighbor we have ', final_ctps.shape[0], ' sources')


# #save final_ctps_csv to csv
final_ctps_csv = final_ctps[['srcname_fin', 'desi_id']]
final_ctps_csv.rename(columns={'desi_id':'desi_id_true'}, inplace=True)
final_ctps_csv.desi_id_true = 'hostless'
final_ctps_csv.to_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv', index=False)
print('Final number of hostless', final_ctps.shape[0])


After merging XMM and CSC and deleting duplicates we have  30  sources
After removing sources with more than one X-ray neighbor we have  30  sources
Final number of hostless 30


/var/folders/hk/rctb6p8n16nbrbnjrwzrc8680000gp/T/ipykernel_45262/2365644178.py:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  final_ctps = pd.concat([xmm_desi, csc_desi])


In [11]:
#combine with non-hostless list
df_ctps = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_no_hostless.csv')
df_hostless = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv')
df_ctps = pd.concat([df_ctps, df_hostless])

df_ctps.to_csv(data_path+'validation_ctps_ero_desi_lh.csv', index = False)
df_ctps.sample(10)

,srcname_fin,desi_id_true
448,SRGe J105216.0+574110,9011_610725_2682
249,SRGe J104510.2+592824,9011_615996_3602
66,SRGe J103413.7+595948,9011_617450_3253
4,SRGe J103143.5+573156,9011_609939_1343
503,SRGe J105553.0+573152,9011_609952_1164
247,SRGe J104506.8+560034,9011_605185_4073
398,SRGe J105003.3+571852,9011_609169_134
452,SRGe J105234.1+552839,9011_603561_2905
111,SRGe J103604.1+562318,9011_606788_1766
212,SRGe J104351.3+585725,9011_614515_1819
